### Install the libraries

In [1]:
# !pip install pinecone llama-index-vector-stores-pinecone llama-index-core llama-index-embeddings-huggingface -q
# pip install -U llama-index-core llama-index-embeddings-huggingface
!pip install -U langchain langchain-core langchain-openai langgraph python-dotenv openai pandas llama-index llama-index-vector-stores-pinecone llama-index-core llama-index-embeddings-huggingface llama-index-llms-groq



  Using cached openai-2.6.1-py3-none-any.whl.metadata (29 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import the libraries

In [2]:
from llama_index.core import(
    Settings, 
    VectorStoreIndex, 
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
import time


c:\my_data_drive\EXL_Training\ma_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize the APIs

In [3]:
# loading the .env file
load_dotenv()

# initializing Groq and Pinecone APIs
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key = PINECONE_API_KEY)

### LLM and Embedding Model Configuration

In [4]:
llm = Groq(model = "llama-3.1-8b-instant")
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

### Pinecone Vector DB Creation

In [5]:
# setting index_name
index_name = "holiday-ads-llama3-groq"

# checking existing indexes
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# creating an index if it doesn't exists
if index_name not in existing_indexes: 
    pc.create_index(
        name = index_name, 
        dimension = 768, 
        metric = "cosine", 
        spec = ServerlessSpec(
                cloud = "aws", 
                region = "us-east-1"
        )
    )

# connecting to the created index
index = pc.Index(index_name)
time.sleep(1)

# describing index statistics
index.describe_index_stats()



{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

### Loading the data

In [6]:
documents = documents = SimpleDirectoryReader("store_offers_data").load_data()

### Using Pinecone Vector Store

In [7]:
vector_store = PineconeVectorStore(pinecone_index = index)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

Upserted vectors: 100%|██████████| 6/6 [00:01<00:00,  4.36it/s]


### Querying the stored data

In [ ]:
question = f"Are there any offers in January for groceries, If yes tell me"

query_engine = index.as_query_engine()
response = query_engine.query(f"{question}")

2025-10-28 01:51:27,353 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [15]:
print(response)

Yes, there are offers in January for groceries. 

You can enjoy 15% off on all health-conscious groceries from January 1 to January 15 with the "New Year Fresh Start" offer.


In [20]:
# If we directly need to call the indexed data:
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone
from dotenv import load_dotenv
import os

# Load API Keys
load_dotenv()
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

# Initialize Groq + Embedding
llm = Groq(model="llama-3.1-8b-instant")
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

# Connect to Existing Pinecone Index
index_name = "holiday-ads-llama3-groq"
pc = Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pc.Index(index_name)

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the existing index (no re-embedding)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context
)

# Query the RAG System
query_engine = index.as_query_engine(similarity_top_k=5)
# question = "What are the offers available for groceries in October?"
question = "What are the offers available for new year?"
response = query_engine.query(question)

print("\nQuery:")
print(question)
print("\nResponse:")
print(response)


2025-10-28 02:01:18,196 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2025-10-28 02:01:24,344 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-10-28 02:01:24,346 - INFO - Retrying request to /chat/completions in 24.000000 seconds
2025-10-28 02:01:48,901 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-28 02:01:48,937 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-10-28 02:01:48,937 - INFO - Retrying request to /chat/completions in 6.000000 seconds
2025-10-28 02:01:55,226 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-28 02:01:55,261 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-10-28 02:01:55,262 - INFO - Retrying request to /chat/completions i


Query:
What are the offers available for new year?

Response:
**Repeat** 

There are two offers available for the New Year:

1. Enjoy 15% off on all health-conscious groceries when you stock your pantry with HarvestNest Spring Essence Organic Quinoa and SunnyHarvest Tropical Bliss Fruit Medley from January 1 to January 15.
2. Enjoy an exclusive 15% off all kits for a limited time with the SpringCloud Adventure Kit from January 1 to January 20.
